In [ ]:
from azureml.core import Workspace, Experiment


project_title = "udacity-project"
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name=project_title)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

cluster_name = "cluster-vhcg77"
# verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name = cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes = 1, idle_seconds_before_scaledown=120)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--solver': choice('newton-cg', 'lbfgs', 'liblinear'),
    '--penalty': choice('none', 'l1', 'l2', 'elasticnet'),
    '--C': choice(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100),
    '--max_iter': choice(100, 200, 300, 400, 500)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', compute_target=cpu_cluster, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig( hyperparameter_sampling=ps, 
                                        primary_metric_name='accuracy', 
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                        max_total_runs=100, 
                                        max_concurrent_runs=4, 
                                        max_duration_minutes=10080, 
                                        policy=policy, 
                                        estimator=est)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
print('completed')

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

best_model = best_run.register_model(model_name='udacity-project')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(dataset_path)

In [ ]:
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.data.dataset_factory import TabularDatasetFactory

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

cleaned_dataset = pd.concat([x,y], axis=1)

# Split data into train and test data
train_dataset, test_dataset = train_test_split(cleaned_dataset, test_size=0.2, random_state=42)

# Export data as csv
train_dataset.to_csv("train/train_data.csv", index=False)
test_dataset.to_csv("test/test_data.csv", index=False)

# Upload data to the datastore
datastore = ws.get_default_datastore()
datastore.upload(src_dir='./train', target_path = project_title)
datastore.upload(src_dir='./test', target_path = project_title)


train_dataset_azure = TabularDatasetFactory.from_delimited_files(path=datastore.path("udacity-project/train_data.csv"))
test_dataset_azure = TabularDatasetFactory.from_delimited_files(path=datastore.path("udacity-project/test_data.csv"))



In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
                                experiment_timeout_minutes=30,
                                task='classification',
                                primary_metric='accuracy',
                                training_data=train_dataset_azure,
                                label_column_name='y',
                                n_cross_validations=5)

In [2]:
# Submit your automl run

run = exp.submit(config=automl_config, show_output=True)
run.wait_for_completion()

In [ ]:
# Retrieve and save your best automl model.

best_automl_run, fitted_automl_model = run.get_output()
print(best_automl_run)
print(fitted_automl_model)

joblib.dump(fitted_automl_model, 'out/fitted_model.joblib')

cpu_cluster.delete()